In [ ]:
import sys
import pydicom
from IPython.display import clear_output
import time
import math
from IPython.display import clear_output
from pydicom.data import get_testdata_file
from pydose_rt.data import MachineConfig
# from pydose_rt.data import MachineConfig
import numpy as np
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
from scipy.ndimage import zoom, rotate
from pydose_rt import DoseEngine
from pydose_rt.utils.plotting import overlay_mask_outline
from pydose_rt.data.loaders import load_dicom
from skimage import measure
import SimpleITK as sitk
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

In [ ]:
roi_colors = {
    "PTV": "orange",
    "ROI1": "red",
    "ROI2": "green",
    "ROI3": "blue",
    "ROI4": "purple",
    "ROI5": "brown",
    "ROI6": "black",
}


In [ ]:
# Set paths
ct_folder = ""
rtplan_path = ""
rtdose_path = ""
rtstruct_path = ""
# Load CT

patient, beam_sequence = load_dicom(ct_folder, dose_path=[rtdose_path], plan_path=[rtplan_path], struct_path=rtstruct_path, struct_names=["CTV", "External"])
beam_sequence = beam_sequence[0]

In [ ]:
plt.subplot(121)
plt.imshow(patient._ct_tensor[110, :, :].cpu().detach().numpy(), cmap='gray')
plt.imshow(patient.dose[110, :, :].cpu().detach().numpy(), cmap='jet', alpha=0.6)
plt.imshow(patient.structures["CTV"][110, :, :].cpu().detach().numpy(), cmap='gray', alpha=0.8)
plt.show()

In [ ]:
ct_slices = np.array(np.expand_dims(patient._ct_tensor.cpu().detach().numpy(), 0), dtype=np.float32)
leafs_1 = beam_sequence.leaf_positions
mus_1 = beam_sequence.mus
plt.figure(figsize=(12, 10))
plt.axis('off')
plt.subplot(311)
plt.title('MLC centers')
plt.imshow(np.transpose(torch.tensor(np.array(leafs_1.cpu().detach().numpy()), dtype=torch.float32, device='cuda').cpu().detach().numpy()[:, :, 0]), cmap='gray', aspect=0.5, interpolation='none')
plt.subplot(312)
plt.title('MLC widths')
plt.imshow(np.transpose(torch.tensor(np.array(leafs_1.cpu().detach().numpy()), dtype=torch.float32, device='cuda').cpu().detach().numpy()[:, :, 1]), cmap='gray', aspect=0.5, interpolation='none')
plt.subplot(313)
plt.title('MUs')
plt.plot(np.linspace(0, 1, 179), torch.tensor(np.array(mus_1.cpu().detach().numpy()), dtype=torch.float32, device='cuda').cpu().detach().numpy()) #, cmap='gray', aspect=10.0, interpolation='none')
plt.show()

In [ ]:
config = MachineConfig(preset="../src/pydose_rt/data/machine_presets/umea_10MV.json")
dose_engine = DoseEngine(
    config, 
    25,
    patient.resolution,
    patient.density_image.shape,
    beam_sequence,
    dtype=torch.float32,
    device=device
    )

dose_1 = dose_engine.compute_dose_sequential(beam_sequence.to(device).to(dtype), density_image=patient.density_image.to(device).to(dtype)).cpu().detach().numpy()

ct_shape = patient._ct_tensor.shape
ext_mask = patient.structures["External"].cpu().detach().numpy() > 0
sag_slice = ct_shape[2] // 2
ax_slice = 71
cor_slice = ct_shape[1] // 2 + 2
dose_pred = dose_1
dose_pred = ext_mask * dose_pred
dose_pred = dose_pred * (np.quantile(patient.dose.cpu().detach().numpy(), 0.999) / np.quantile(dose_pred, 0.999))
diff = ext_mask * np.abs(patient.dose.cpu().detach().numpy() - dose_pred)**2
print(np.mean(diff))

dose_plot = dose_pred
pred_max = np.max(patient.dose.cpu().detach().numpy())
dose_max = np.max(patient.dose.cpu().detach().numpy())
ct_volume = patient._ct_tensor.cpu().detach().numpy()
dose_volume = patient.dose.cpu().detach().numpy()
masks = patient.structures
alpha = 0.9
plt.figure(figsize=(20, 30))
plt.subplot(321)
plt.imshow(ct_volume[:, :, sag_slice], cmap='gray')
plt.imshow(dose_volume[:, :, sag_slice], cmap='jet', vmax=dose_max, alpha=alpha)
for roi_name in masks.keys():
    index = list(masks.keys()).index(roi_name)
    color = list(roi_colors.values())[index]
    roi = masks[roi_name]
    overlay_mask_outline(roi.cpu().detach().numpy()[:, :, sag_slice], color=color)
plt.colorbar()
plt.subplot(322)
plt.imshow(ct_volume[:, :, sag_slice], cmap='gray')
plt.imshow(dose_plot[0, :, :, sag_slice], cmap='jet', vmax=pred_max, alpha=alpha)
for roi_name in masks.keys():
    index = list(masks.keys()).index(roi_name)
    color = list(roi_colors.values())[index]
    roi = masks[roi_name]
    overlay_mask_outline(roi.cpu().detach().numpy()[:, :, sag_slice], color=color)
plt.colorbar()
plt.subplot(323)
plt.imshow(ct_volume[:, cor_slice, :], cmap='gray')
plt.imshow(dose_volume[:, cor_slice, :], cmap='jet', vmax=dose_max, alpha=alpha)
for roi_name in masks.keys():
    index = list(masks.keys()).index(roi_name)
    color = list(roi_colors.values())[index]
    roi = masks[roi_name]
    overlay_mask_outline(roi.cpu().detach().numpy()[:, cor_slice, :], color=color)
plt.colorbar()
plt.subplot(324)
plt.imshow(ct_volume[:, cor_slice, :], cmap='gray')
plt.imshow(dose_plot[0, :, cor_slice, :], cmap='jet', vmax=pred_max, alpha=alpha)
for roi_name in masks.keys():
    index = list(masks.keys()).index(roi_name)
    color = list(roi_colors.values())[index]
    roi = masks[roi_name]
    overlay_mask_outline(roi.cpu().detach().numpy()[:, cor_slice, :], color=color)
plt.colorbar()
plt.subplot(325)
plt.imshow(ct_volume[ax_slice, :, :], cmap='gray')
plt.imshow(dose_volume[ax_slice, :, :], cmap='jet', vmax=dose_max, alpha=alpha)
for roi_name in masks.keys():
    index = list(masks.keys()).index(roi_name)
    color = list(roi_colors.values())[index]
    roi = masks[roi_name]
    overlay_mask_outline(roi.cpu().detach().numpy()[ax_slice, :, :], color=color)
plt.colorbar()
plt.subplot(326)
plt.imshow(ct_volume[ax_slice, :, :], cmap='gray')
plt.imshow(dose_plot[0, ax_slice, :, :], cmap='jet', vmax=pred_max, alpha=alpha)
for roi_name in masks.keys():
    index = list(masks.keys()).index(roi_name)
    color = list(roi_colors.values())[index]
    roi = masks[roi_name]
    overlay_mask_outline(roi.cpu().detach().numpy()[ax_slice, :, :], color=color)
plt.colorbar()
plt.show()